In [1]:
!which python

/home/kamal/Downloads/onboarding/scibert-procodex/.venv/bin/python


In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
with open("../data/raw/20230821_full_data.pkl", "rb") as handle:
    data = pickle.load(handle)

In [4]:
data

,id,title,keywords,content,target
0,7OT5chJU5Y1L6A,"Electromagnetic Compatibility Regulations, 2016",Electromagnetic compatibility;EMC;Radio freque...,VERŻJONI ELETTRONIKA\nB\n\n1277\n\nB 1277\nSup...,Relevant
1,QXU5clgY5Y1L6A,Regulation governing the implementation of cha...,Construction of buses,"Netherlands: RV 2009 Chap. 3, 5, 6;\nApp. IIIA...",Relevant
2,SsT5cxDw5Y1L6A,Traffic Implementation Regulations - Electrom...,Electromagnetic compatibility;EMC;Radio freque...,Österreich: KDV § 17b\n\nElektromagnetische Ve...,Relevant
3,tLf5surH5Y1L6A,NHTSA Federal Automated Vehicles Policy (FAVP)...,Autonomous driving,AUTOMATED DRIVING SYSTEMS\nA Vision for Safety...,Relevant
4,KxZ5sjo75Y1L6A,Decree of the Minister of Equipment and Transp...,Flammability of materials,Morocco: Arr. 2915-10\n\nFlammability of mater...,Relevant
...,...,...,...,...,...
2376,7wpFaHSu5Y1L6A,Luxembourg: New European Standards Applicable ...,Aerospace Products;Building / Construction Pro...,JOURNAL OFFICIEL\nDU GRAND-DUCHÉ DE LUXEMBOURG...,Not relevant
2377,8QkF$3ly5Y1L6A,Luxembourg: New European Standards Applicable ...,Declaring/certifying conformity;Radio/Wireless...,JOURNAL OFFICIEL\nDU GRAND-DUCHÉ DE LUXEMBOURG...,Not relevant
2378,zGpNEAGB5Y1L6A,Lithuania: Implementation of Directive (EU) 20...,Accessibility;Computers;Consumer Protection;De...,LIETUVOS RESPUBLIKOS\nGAMINIŲ IR PASLAUGŲ PRIE...,Not relevant
2379,K2jJ$ks45Y1L6A,"USA: Event Data Recorders for Vehicles, Rules,...",Automotive;Retention/Destruction of records,"Federal Register / Vol. 87, No. 119 / Wednesda...",Not relevant


In [55]:
import os
import re
import string
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

import warnings 
warnings.filterwarnings('ignore')

# 1. lower case
def lower_case(text):
    return text.lower()

# 2. remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# 3. remove stop words
def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_text)

# 4. remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# 5. remove short words
def remove_short_words(text):
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if len(w) > 2]
    return ' '.join(filtered_text)

# 6. lemmatize
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized_text = [lemmatizer.lemmatize(w) for w in word_tokens]
    return ' '.join(lemmatized_text)

# 8. remove non-ascii characters
def remove_non_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

# 9. remove extra spaces
def remove_extra_spaces(text):
    return re.sub(' +', ' ', text)

# 16. remove extra non-breaking spaces
def remove_extra_non_breaking_spaces(text):
    text = re.sub('\xa0+', '', text)
    text = re.sub('\x0c', '', text)

    return text

def remove_dashes_underscores(text):
    text = text.replace("-","")
    text=text.replace("_","")

    return text

def remove_special_characters(text):
    # Define a regular expression pattern to match special characters
    pattern = r'[^a-zA-Z0-9\s]'  # This pattern matches anything that's not alphanumeric or whitespace

    # Use re.sub() to replace matched patterns with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text
    
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Use re.sub() to replace matched URLs with an empty string
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

In [41]:
def process_target(label):
    label_map={
            "Relevant":1,
            "Not relevant":0
    }
    return label_map[label]



def preprocess(text):
    pass

In [42]:
test = pd.read_excel("../data/raw/test_ids.xlsx")
test_ids = list(test["id"].values)

train = data[~data["id"].isin(test_ids)]
test = data[data["id"].isin(test_ids)]

In [57]:
x = train["content"][200].replace("\n", " ")
x = remove_urls(x)
x = remove_non_ascii(x)
x = remove_extra_non_breaking_spaces(x)
x = remove_dashes_underscores(x)
x = remove_special_characters(x)
x = remove_extra_spaces(x)
x

'11112020 Mevzuat B lg S stem TELSIZ EKIPMANLARI YONETMELIGI 201453AB BIRINCI BOLUM Amac Kapsam Dayanak ve Tanmlar Amac MADDE 1 1 Bu Yonetmel g n amac tels z ek pmanlarnn temel gereklere uygun olarak p yasaya arz ed lmes p yasada bulundurulmas ve h zmete sunulmasna l sk n kt sad sletmec ler n yukumlulukler tels z ek pmanlarnn uygunlugu uygunluk degerlend rme kuruluslar le p yasa gozet m ve denet m le lg l usul ve esaslar duzenlemekt r Kapsam MADDE 2 1 Bu Yonetmel k Devlet ve kamu guvenl g savunma ve Devlet n ceza hukuku alanndak faal yetler nde kullanlan ozel amacl ek pmanlar le Ek1de bel rt len ek pmanlar har c tum tels z ek pmanlarn kapsar 2 Bu Yonetmel k kapsamna g ren tels z ek pmanlarna 5 nc madden n b r nc fkrasnn a bend nde bel rt len hususlar har c 2102016 tar hl ve 29845 sayl Resmi Gazetede yaymlanan Bel rl Ger l m Snrlar Ic n Tasarlanan Elektr kl Ek pman le Ilg l Yonetmel k 201435AB hukumler uygulanmaz Dayanak MADDE 3 1 Bu Yonetmel k 5112008 tar hl ve 5809 sayl Elektron k Hab